In [398]:
import numpy as np
import pysut
import pandas as pd
import json
import pickle
import os

In [3]:
V = np.array([[1,0,0],[0.5,1,0],[0,0,1]])
U = np.array([[0,0.1,0.4],[0,0,1],[0.5,0.5,0]])
g = V.sum(axis=0)
q = V.sum(axis=1)

In [4]:
print("V:")
print(V)
print("U:")
print(U)
print("g:")
print(g)
print("q:")
print(q)

V:
[[1.  0.  0. ]
 [0.5 1.  0. ]
 [0.  0.  1. ]]
U:
[[0.  0.1 0.4]
 [0.  0.  1. ]
 [0.5 0.5 0. ]]
g:
[1.5 1.  1. ]
q:
[1.  1.5 1. ]


$A_\mathrm{ITC-cxc} = U \cdot \hat{g}^{-1} \cdot V' \cdot \hat{q}^{-1}$

In [5]:
U.dot(np.diag(1/g)).dot((V.T).dot(np.diag(1/q)))

array([[0.        , 0.06666667, 0.4       ],
       [0.        , 0.        , 1.        ],
       [0.33333333, 0.44444444, 0.        ]])

In [6]:
SUT = pysut.SupplyUseTable(V=V, U=U)

In [7]:
SUT.build_mr_Gamma()

In [8]:
SUT.build_E_bar()

In [9]:
A, __, __, __, Z, __ = SUT.aac_agg(keep_size=False)

residual: -5.00e-01
residual: +0.00e+00
number of iterations:3


In [10]:
print(Z)

[[-0.05  0.15  0.4 ]
 [ 0.    0.    1.  ]
 [ 0.25  0.75  0.  ]]


In [11]:
SUT.E_bar

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [12]:
SUT.Gamma

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [13]:
V_A = np.array([[1,0,0],[0.6,0,0],[0,0,1]])
V_B = np.array([[1,0,0],[0,1,0],[0,0,1]])
U_A = np.array([[0,0,0.5],[0,0,0.5],[0.5,0,0]])
U_B = np.array([[0,0.3,0],[1,0,0],[0.5,0,0]])

V_AB = np.array([[0,0,0],[0,0,0],[0,0,0]])
V_BA = np.array([[0,0,0],[0,0,0],[0,0,0]])
U_AB = np.array([[0,0,0],[0,0,0],[0.5,0,0]])
U_BA = np.array([[0.1,0,0],[0,0,0],[0,0,0]])

In [14]:
V_total = np.concatenate((np.concatenate((V_A,V_AB), axis=1),np.concatenate((V_BA,V_B),axis=1)),axis=0)
U_total = np.concatenate((np.concatenate((U_A,U_AB), axis=1),np.concatenate((U_BA,U_B),axis=1)),axis=0)
V_total

array([[1. , 0. , 0. , 0. , 0. , 0. ],
       [0.6, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. ]])

In [15]:
U_total

array([[0. , 0. , 0.5, 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5, 0. , 0. ],
       [0.1, 0. , 0. , 0. , 0.3, 0. ],
       [0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. ]])

In [16]:
SUT_MR = pysut.SupplyUseTable(V = V_total, U=U_total, regions=2)

In [17]:
SUT_MR.build_mr_Gamma()
SUT_MR.build_E_bar()
SUT_MR.build_mr_Xi()
#A_MR,_,_,_,_,_,Z_MR,_ = SUT_MR.psc_agg()

/home/jakobs/anaconda3/lib/python3.6/site-packages/pysut/pySUT.py:1776: RuntimeWarning: divide by zero encountered in true_divide
  y = 1 / x


In [18]:
print(SUT_MR.Xi)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [19]:
SUT_MR.aac_agg(keep_size=False)

residual: -6.00e-01
residual: +0.00e+00
number of iterations:3


/home/jakobs/anaconda3/lib/python3.6/site-packages/pysut/pySUT.py:1776: RuntimeWarning: divide by zero encountered in true_divide
  y = 1 / x


(array([[ 0.  ,  0.  ,  0.5 ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.5 ,  0.  ,  0.  ,  0.  ],
        [ 0.5 ,  0.  ,  0.  ,  0.5 ,  0.  ,  0.  ],
        [-0.08,  0.3 ,  0.  ,  0.  ,  0.3 ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.5 ,  0.  ,  0.  ]]),
 array([], dtype=float64),
 array([ True,  True,  True,  True,  True,  True]),
 array([ True,  True,  True,  True,  True,  True]),
 array([[ 0.  ,  0.  ,  0.5 ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.5 ,  0.  ,  0.  ,  0.  ],
        [ 0.5 ,  0.  ,  0.  ,  0.5 ,  0.  ,  0.  ],
        [-0.08,  0.18,  0.  ,  0.  ,  0.3 ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.5 ,  0.  ,  0.  ]]),
 array([], dtype=float64))

In [20]:
print("V")
print(V_total)
print("U")
print(U_total)

V
[[1.  0.  0.  0.  0.  0. ]
 [0.6 0.  0.  0.  0.  0. ]
 [0.  0.  1.  0.  0.  0. ]
 [0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  0.  1. ]]
U
[[0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.5 0.  0.  0. ]
 [0.5 0.  0.  0.5 0.  0. ]
 [0.1 0.  0.  0.  0.3 0. ]
 [0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.5 0.  0. ]]


In [21]:
SUT_MR.Gamma

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [22]:
SUT_MR.E_bar

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])

In [23]:
SupplyDiag_Eval = SUT_MR.supply_diag_check()
print(SupplyDiag_Eval)


[[1.  0.  0.  0.  0.  1.  1.6]
 [0.  0.  1.  0.  0.  0.6 0. ]
 [1.  0.  0.  0.  0.  1.  1. ]
 [1.  0.  0.  0.  0.  1.  1. ]
 [1.  0.  0.  0.  0.  1.  1. ]
 [1.  0.  0.  0.  0.  1.  1. ]]


In [24]:
#A_MR,_,_,_,_,_,Z_MR,_ = 
SUT_MR.psc_agg()

(array([[ 0. ,  0. ,  0.5,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0.5,  0. ,  0. ,  0. ],
        [ 0.5,  0. ,  0. ,  0.5,  0. ,  0. ],
        [ 0.1,  0. ,  0. ,  0. ,  0.3,  0. ],
        [-0.6,  0. ,  0. ,  1. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0.5,  0. ,  0. ]]),
 array([[0. , 0. , 0.5, 0. , 0. , 0. ],
        [0. , 0. , 0.5, 0. , 0. , 0. ],
        [0.5, 0. , 0. , 0.5, 0. , 0. ],
        [0.1, 0. , 0. , 0. , 0.3, 0. ],
        [0. , 0. , 0. , 1. , 0. , 0. ],
        [0. , 0. , 0. , 0.5, 0. , 0. ]]),
 array([[0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ],
        [0.6, 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ]]),
 array([], dtype=float64),
 array([ True, False,  True,  True,  True,  True]),
 array([ True, False,  True,  True,  True,  True]),
 array([], dtype=float64),
 array([], dtype=float64))

In [ ]:
print("V")
print(V_total)
print("U")
print(U_total)

SUT_MR.btc()

In [ ]:
np.dot(SUT_MR.Xi,V_total-np.diag(np.diag(V_total)))

In [ ]:
np.dot(SUT_MR.Xi,U_total)

In [ ]:
print("V")
print(V_total)
print("U")
print(U_total)


np.dot(SUT_MR.Xi,U_total)-np.dot(SUT_MR.Xi,V_total-np.diag(np.diag(V_total)))

In [ ]:
SUT_MR.psc_agg()[0]-np.dot(SUT_MR.Xi,U_total)-np.dot(SUT_MR.Xi,V_total-np.diag(np.diag(V_total)))

In [426]:
exio_v = pd.read_csv('/home/jakobs/Documents/IndEcol/BONSAI/data/exiobase_hsut_3317/MR_HSUP_2011_v3_3_17_bonsai.csv', header=[0,1,2,3], index_col=[0,1,2,3,4])

In [427]:
exio_u = pd.read_csv('/home/jakobs/Documents/IndEcol/BONSAI/data/exiobase_hsut_3317/MR_HUSE_2011_v3_3_17_bonsai.csv', header=[0,1,2,3], index_col=[0,1,2,3,4])

In [428]:
exio_u.values.shape

(9600, 7872)

In [429]:
exio_v.values.shape

(9600, 7872)

In [243]:
exio_u.loc[exio_u.index.get_level_values(2) == 'p40.2.b']

AU  \
                                           Cultivation of paddy rice   
                                                               i01.a   
Country code                                                  A_PARI   
AU Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
AT Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
BE Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
BR Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
BG Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
CA Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
CN Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
CY Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
CZ Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
DK Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
EE Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
FI Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
FR Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
DE Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
GR Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
HU Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
HR Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
IN Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
ID Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
IE Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
IT Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
JP Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
LV Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
LT Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
LU Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
MT Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
MX Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
NL Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
NO Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
PL Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
PT Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
RO Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
RU Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
SK Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
SI Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
ZA Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
KR Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
ES Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
SE Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
CH Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
TR Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
GB Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
US Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
WA Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
WL Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
WE Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
WF Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   
WM Blast Furnace Gas p40.2.b C_MBFG tonnes                       0.0   

                                                                 \
                                           Cultivation of wheat   
                                                          i01.b   
Country code                                             A_WHEA   
AU Blast Furnace Gas p40.2.b C_MBFG tonnes           

In [244]:
np.where(exio_u.loc[exio_u.index.get_level_values(2) == 'p40.2.b'].values >0)

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,
         2,  3,  3,  3,  5,  6,  6,  6,  6,  6,  6,  6,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 11, 11, 11, 11, 12,
        12, 12, 12, 13, 13, 13, 13, 15, 15, 15, 17, 20, 20, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 26, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28,
        28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29, 31, 31, 31, 31,
        31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33,
        33, 33, 33, 35, 36, 37, 37, 37, 38, 38, 38, 38, 40, 41, 41, 41, 41,
        41, 42, 42, 42, 42, 42, 43, 43, 43, 43, 44, 44, 45, 45, 45, 45, 45,
        45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,
        45, 45, 45, 45, 46, 47, 47, 47]),
 array([  55,   64,   65,   66,   67,   68,   70,   83,   96,  110,  219,
         247,  260,  274,  275,  411,  424,  439

In [246]:
(exio_v.loc[exio_v.index.get_level_values(1) == 'Biogas'].values >0)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [247]:
exioSUT = pysut.SupplyUseTable(V=exio_v.values, U=exio_u.values)

In [430]:
calval = pd.read_csv('/home/jakobs/Documents/IndEcol/BONSAI/data/exiobase_hsut_3317/Calorific_values.csv', header=[4], index_col=[0,1,2,3])

In [431]:
calval.fillna(1, inplace=True)

In [432]:
aggregation_matrix = pd.read_csv('/home/jakobs/Documents/IndEcol/BONSAI/data/exiobase_hsut_3317/aggregation_matrix_exiobase.csv', header=[0,1,2], index_col=[0,1,2,3])

In [433]:
aggregation_matrix.ix[aggregation_matrix.iloc[:,109]==1,109]

/home/jakobs/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


142  Coke oven gas             p40.2.a  C_COOG    1
143  Blast Furnace Gas         p40.2.b  C_MBFG    1
144  Oxygen Steel Furnace Gas  p40.2.c  C_MOSG    1
145  Gas Works Gas             p40.2.d  C_MGWG    1
146  Biogas                    p40.2.e  C_MBIO    1
Name: (Manufacture of gas;, i40.2.a, A_MGWG), dtype: int64

### Below is a litte function to document the aggregation

In [434]:
N_regs = 48
N_sec = 164
N_prod = 200
nr_of_products = []
testfile = '/home/jakobs/Desktop/testfile.csv'
with open(testfile, 'w') as f:
    f.write('#Agrgegation report \n')
    f.write('|'.join(['Country code','Industry name','Industry code 1','Industry code 2', 'Number of products to be aggregated', 'Product name', 'Product code 1','Product code 2', 'Aggregation Value', '\n']))
    for c in range(N_regs):
        for i in range(aggregation_matrix.shape[1]):
            n_prods = aggregation_matrix.iloc[:,i].sum()
            if n_prods > 1:
                prod_inds = np.where(aggregation_matrix.iloc[:,i]==1)[0]
                #print(prod_inds)
                aggregated_prods = [aggregation_matrix.index[x] for x in prod_inds]
                aggregation_values = new_aggregation_matrix[prod_inds+c*N_prod, i+c*N_sec]
                #print(aggregation_values)
                nr_of_products.append(n_prods)
                #print(*aggregation_matrix.columns[i], n_prods)
                for j,prod in enumerate(aggregated_prods):
                    #if j == 0:
                    #    print(prod[1:])
                        f.write('|'.join([country_list[c],'|'.join(aggregation_matrix.columns[i]), '{}'.format(n_prods), '|'.join(prod[1:]), '{}'.format(aggregation_values[j]), '\n']))
                    #else:
                    #    f.write('|'.join(['','','','','', '|'.join(prod[1:]), '{}'.format(aggregation_values[j]), '\n']))

df = pd.read_csv('/home/jakobs/Desktop/testfile.csv', delimiter='|', skiprows=1, index_col=[0,1])
                            
df


Industry code 1  \
Country code Industry name                                                        
AU           Mining of coal and lignite; extraction of peat ...             i10   
             Mining of coal and lignite; extraction of peat ...             i10   
             Mining of coal and lignite; extraction of peat ...             i10   
             Mining of coal and lignite; extraction of peat ...             i10   
             Mining of coal and lignite; extraction of peat ...             i10   
             Mining of coal and lignite; extraction of peat ...             i10   
             Mining of coal and lignite; extraction of peat ...             i10   
             Mining of coal and lignite; extraction of peat ...             i10   
             Extraction of natural gas and services related ...           i11.b   
             Extraction of natural gas and services related ...           i11.b   
             Manufacture of coke oven products                            i23.1   
             Manufacture of coke oven products                            i23.1   
             Manufacture of coke oven products                            i23.1   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
...                                                                         ...   
WM           Manufacture of coke oven products                            i23.1   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2   
             Petroleum Refinery                                           i23.2 

In [435]:
#aggregation_matrix[aggregation_matrix['Manufacture of gas;']>0]
aggregation_matrix.columns[109][0][:-1]

'Manufacture of gas'

In [436]:
calval.iloc[141:146]

,,,,AU,AT,BE,BR,BG,CA,CN,CY,CZ,DK,...,CH,TW,TR,GB,US,WA,WL,WE,WF,WM
,Unnamed: 1,Unnamed: 2,Unnamed: 3,,,,,,,,,,,,,,,,,,,,,
142,Coke oven gas,p40.2.a,C_COOG,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,...,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870
143,Blast Furnace Gas,p40.2.b,C_MBFG,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,...,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247,0.00247
144,Oxygen Steel Furnace Gas,p40.2.c,C_MOSG,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,...,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706,0.00706
145,Gas Works Gas,p40.2.d,C_MGWG,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,...,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870,0.03870
146,Biogas,p40.2.e,C_MBIO,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,...,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040,0.05040


In [437]:
calval.loc[calval.index.get_level_values(3) == 'C_GASE']

,,,,AU,AT,BE,BR,BG,CA,CN,CY,CZ,DK,...,CH,TW,TR,GB,US,WA,WL,WE,WF,WM
,Unnamed: 1,Unnamed: 2,Unnamed: 3,,,,,,,,,,,,,,,,,,,,,
29,"Natural gas and services related to natural gas extraction, excluding surveying",p11.b,C_GASE,0.048,0.048,0.048,0.048,0.048,0.048,0.048,0.048,0.048,0.048,...,0.048,0.048,0.048,0.048,0.048,0.048,0.048,0.048,0.048,0.048


In [438]:
Natural_gas_calval = calval.loc[calval.index.get_level_values(3) == 'C_GASE'].values[0] #natural gas production has the same caloric value for now but we keep the a specific value for now
print(Natural_gas_calval)
calval.iloc[141:146]/Natural_gas_calval


[0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048
 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048
 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048
 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048 0.048]


,,,,AU,AT,BE,BR,BG,CA,CN,CY,CZ,DK,...,CH,TW,TR,GB,US,WA,WL,WE,WF,WM
,Unnamed: 1,Unnamed: 2,Unnamed: 3,,,,,,,,,,,,,,,,,,,,,
142,Coke oven gas,p40.2.a,C_COOG,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,...,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250
143,Blast Furnace Gas,p40.2.b,C_MBFG,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,...,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458,0.051458
144,Oxygen Steel Furnace Gas,p40.2.c,C_MOSG,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,...,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083,0.147083
145,Gas Works Gas,p40.2.d,C_MGWG,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,...,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250,0.806250
146,Biogas,p40.2.e,C_MBIO,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,...,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000,1.050000


In [439]:
new_aggregation_matrix = np.zeros((48*200,48*164))
for i in range(48):
    new_aggregation_matrix[i*200:i*200+200,i*164:i*164+164] = aggregation_matrix

In [440]:
new_aggregation_matrix[200,164]

1.0

In [441]:
for i in range(48):
    #print(calval.iloc[141:146,i]/Natural_gas_calval[i])
    new_aggregation_matrix[i*200+141:i*200+146,i*164+109] = calval.iloc[141:146,i]/Natural_gas_calval[i]

In [442]:
j=10
new_aggregation_matrix[141+j*200,109+164*j]

0.8062499999999999

In [443]:
exio_vagg = new_aggregation_matrix.T.dot(exio_v.values)

In [444]:
j=1
print(exio_vagg[24+j*164,24+j*164])
exio_vagg.shape

706211.2


(7872, 7872)

In [445]:
exio_v.loc[exio_v.index.get_level_values(3)=='C_IRON','AU'].values

/home/jakobs/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [446]:
aggregation_matrix

,,,,Cultivation of paddy rice,Cultivation of wheat,Cultivation of cereal grains nec,"Cultivation of vegetables, fruit, nuts",Cultivation of oil seeds,"Cultivation of sugar cane, sugar beet",Cultivation of plant-based fibers,Cultivation of crops nec,Cattle farming,Pigs farming,...,Landfill of waste: Paper,Landfill of waste: Plastic,Landfill of waste: Inert/metal/hazardous,Landfill of waste: Textiles,Landfill of waste: Wood,Activities of membership organisation n.e.c. (91),"Recreational, cultural and sporting activities (92)",Other service activities (93),Private households with employed persons (95),Extra-territorial organizations and bodies
,,,,i01.a,i01.b,i01.c,i01.d,i01.e,i01.f,i01.g,i01.h,i01.i,i01.j,...,i90.6.b,i90.6.c,i90.6.d,i90.6.e,i90.6.f,i91,i92,i93,i95,i99
,,,,A_PARI,A_WHEA,A_OCER,A_FVEG,A_OILS,A_SUGB,A_FIBR,A_OTCR,A_CATL,A_PIGS,...,A_LANP,A_LANL,A_LANI,A_LANT,A_LANW,A_ORGA,A_RECR,A_OSER,A_PRHH,A_EXTO
1,Paddy rice,p01.a,C_PARI,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Wheat,p01.b,C_WHEA,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cereal grains nec,p01.c,C_OCER,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Vegetables, fruit, nuts",p01.d,C_FVEG,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Oil seeds,p01.e,C_OILS,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Sugar cane, sugar beet",p01.f,C_SUGB,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Plant-based fibers,p01.g,C_FIBR,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Crops nec,p01.h,C_OTCR,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [447]:
exio_uagg = new_aggregation_matrix.T.dot(exio_u.values)

In [448]:
exio_SUTagg_Obj = pysut.SupplyUseTable(V=exio_vagg, U=exio_uagg, regions=48)

In [449]:
iot_names = pd.read_csv('/home/jakobs/data/EXIOBASE/EXIOBASE_3.3.17_hsut_2011/Classifications_v_3_3_17_IOTnames.csv')

country_list = iot_names['Country code'].unique()
country_dic = {}
for i,c in enumerate(country_list):
    country_dic[c] = i


products = iot_names[['Product name', 'Product code 1']]
products.drop_duplicates(inplace=True)

prod_dic = {}
for i,prod in enumerate(products['Product name'].values):
    prod_dic[prod] = i

/home/jakobs/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [450]:
products[products['Product name'].str.contains('Distribution')]

,Product name,Product code 1
108,Distribution and trade services of electricity,p40.13
110,Distribution services of gaseous fuels through...,p40.2.2


In [451]:
exio_vagg.shape

(7872, 7872)

In [452]:
supply_diag_check_eval = exio_SUTagg_Obj.supply_diag_check()
supply_diag_check_eval.shape
prod_names = iot_names.values[supply_diag_check_eval[:,2]==1,:]
print(prod_names)

[['AT' 'Wool, silk-worm cocoons' 'p01.o' 'C_WOOL' 'tonnes']
 ['AT' 'Aluminium and aluminium products' 'p27.42' 'C_ALUM' 'tonnes']
 ['AT' 'Biogas an other gases nec.' 'p40.2.1' 'C_BGAS' 'tonnes']
 ['BE' 'Wool, silk-worm cocoons' 'p01.o' 'C_WOOL' 'tonnes']
 ['BE' 'Aluminium and aluminium products' 'p27.42' 'C_ALUM' 'tonnes']
 ['BE' 'Biogas an other gases nec.' 'p40.2.1' 'C_BGAS' 'tonnes']
 ['BR' 'Wool, silk-worm cocoons' 'p01.o' 'C_WOOL' 'tonnes']
 ['BR' 'Biogas an other gases nec.' 'p40.2.1' 'C_BGAS' 'tonnes']
 ['BG' 'Wool, silk-worm cocoons' 'p01.o' 'C_WOOL' 'tonnes']
 ['BG' 'Aluminium and aluminium products' 'p27.42' 'C_ALUM' 'tonnes']
 ['BG' 'Electricity by biomass and waste' 'p40.11.g' 'C_POWB' 'TJ']
 ['CA' 'Wool, silk-worm cocoons' 'p01.o' 'C_WOOL' 'tonnes']
 ['CA' 'Biogas an other gases nec.' 'p40.2.1' 'C_BGAS' 'tonnes']
 ['CN' 'Wool, silk-worm cocoons' 'p01.o' 'C_WOOL' 'tonnes']
 ['CY' 'Wool, silk-worm cocoons' 'p01.o' 'C_WOOL' 'tonnes']
 ['CY' 'N-fertiliser' 'p24.b' 'C_NFER' 'to

In [479]:
def create_market_and_product_names(prod_names, N_reg, Reg_list):                          
    """Create market names, and product names.                                   
    Input:                                                                       
    prod_names      :   array of exclusive byproducts (this should include at    
                        at least one electricity byproduct e.g. electricity      
                        from coal (as byproduct from heat from coal).            
    N_reg           :   number of regions in the MRIO system                     
                                                                                 
    Output:                                                                      
    excl_byproducts :   array of exclusive byproducts (excl electricity)         
    market_names    :   array of market names corresponding to the excl byprods  
    grid_electricity:   array of 'Electricity from grid' names for the different 
                        regions. i.e. 1 entry for each electricity market        
    elec_markets    :   Like grid_electricity but now "Market for electricity"   
                                                                                 
    All outputs have the format [['Region', 'Name', 'code 1', 'code 2', 'unit']] 
    where the unit only exists for the products not for the markets.             
    """                                                                          
    #get a unique list of the product names to create global markets             
    unique_prod_indices = np.unique(prod_names[:,1], return_index=True)[-1]      
    excl_byproducts = prod_names[unique_prod_indices]                            
    #print(market_names)                                                         
    market_names = excl_byproducts.copy()                                        
    market_names[:,0] = 'GLO' #set the region to global                          
    #now change the names of the products to Market for 'product', incl codes.   
    #also remove the electricity from the excl_byproducts                        
    rm_index = []                                                                
    for i,prod in enumerate(excl_byproducts):                               
        market_names[i,1] = 'Market for ' + market_names[i,1]                    
        market_names[i,2] = market_names[i,2].replace('p','m')                   
        market_names[i,3] = market_names[i,3].replace('C_','M_')                 
        #there are multiple electricity byproducts. they supplyt the same market 
        #so give them the same code/market name                                  
        if 'm40.11' in market_names[i,2]:                                        
            market_names[i,2] = 'm40.11'                                         
            market_names[i,1] = 'Market for electricity'                         
            market_names[i,3] = 'M_ELEC'                                         
        if 'p40.11.' in prod[2]:                                                 
            rm_index.append(i)                                                   
    #remove electricity from excl byproducts                                     
    excl_byproducts = np.delete(excl_byproducts, np.array(rm_index), axis=0)     
    #drop the duplicate electricity markets                                      
    market_name_indices = np.unique(market_names[:,1], return_index=True)[-1]    
    market_names = market_names[market_name_indices]                             
    #split the markets into exclusive byproduct markets and electricity markets  
    elec_markets = np.array([x for x in market_names if x[2]=='m40.11']*N_reg).reshape(N_reg,5) 
    elec_markets[:,0] = Reg_list
    market_names = np.delete(market_names,                                       
                             np.where(market_names[:,2]=='m40.11')[0], axis=0)   
    #create electricity market products ('Electricity from the grid')            
    grid_electricity = elec_markets.copy()                                       
    for i in range(len(elec_markets)):                                           
        grid_electricity[i,1] = 'Electricity from the grid'                      
        grid_electricity[i,2] = grid_electricity[i,2].replace('m','p')           
        grid_electricity[i,3] = grid_electricity[i,3].replace('M_','C_',)        
    #drop units for markets:                                                     
    elec_markets = elec_markets[:,:-1]                                           
    market_names = market_names[:,:-1]                                           
    return excl_byproducts, market_names, grid_electricity, elec_markets

In [480]:
create_market_and_product_names(prod_names, 48, country_list)

(array([['AT', 'Aluminium and aluminium products', 'p27.42', 'C_ALUM',
         'tonnes'],
        ['AT', 'Biogas an other gases nec.', 'p40.2.1', 'C_BGAS',
         'tonnes'],
        ['CY', 'N-fertiliser', 'p24.b', 'C_NFER', 'tonnes'],
        ['GR', 'Oil seeds', 'p01.e', 'C_OILS', 'tonnes'],
        ['CY', 'P- and other fertiliser', 'p24.c', 'C_PFER', 'tonnes'],
        ['MT', 'Plastics, basic', 'p24.a', 'C_PLAS', 'tonnes'],
        ['MT', 'Pulp', 'p21.1', 'C_PULP', 'tonnes'],
        ['AT', 'Wool, silk-worm cocoons', 'p01.o', 'C_WOOL', 'tonnes']],
       dtype=object),
 array([['GLO', 'Market for Aluminium and aluminium products', 'm27.42',
         'M_ALUM'],
        ['GLO', 'Market for Biogas an other gases nec.', 'm40.2.1',
         'M_BGAS'],
        ['GLO', 'Market for N-fertiliser', 'm24.b', 'M_NFER'],
        ['GLO', 'Market for Oil seeds', 'm01.e', 'M_OILS'],
        ['GLO', 'Market for P- and other fertiliser', 'm24.c', 'M_PFER'],
        ['GLO', 'Market for Plastics, basi

In [468]:
market_name_indices = np.unique(prod_names[:,1], return_index=True)[-1]

market_names = prod_names[market_name_indices]
#print(market_names)
market_names[:,0] = 'GLO'
for i in range(len(market_names)):
    market_names[i,1] = 'Market for ' + market_names[i,1]
    market_names[i,2] = market_names[i,2].replace('p','m')
    market_names[i,3] = market_names[i,3].replace('C_','M_')
    if 'm40.11' in market_names[i,2]:
        market_names[i,2] = 'm40.11'
        market_names[i,1] = 'Market for electricity'
        market_names[i,3] = 'M_ELEC'

market_name_indices = np.unique(market_names[:,1], return_index=True)[-1]
market_names = market_names[market_name_indices]

elec_markets = np.array([x for x in market_names if x[2]=='m40.11']*48).reshape(48,5)
elec_markets[:,0] = iot_names['Country code'].unique()
market_names = np.delete(market_names, np.where(market_names[:,2]=='m40.11')[0], axis=0)
#np.append(market_names, elec_markets, axis=0)
print(elec_markets)
market_names

[['AU' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['AT' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['BE' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['BR' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['BG' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['CA' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['CN' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['CY' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['CZ' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['DK' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['EE' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['FI' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['FR' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['DE' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['GR' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['HU' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['HR' 'Market for electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['IN' 'Market for electricity' 'm40.11' 'M_ELEC

array([['GLO', 'Market for Aluminium and aluminium products', 'm27.42',
        'M_ALUM', 'tonnes'],
       ['GLO', 'Market for Biogas an other gases nec.', 'm40.2.1',
        'M_BGAS', 'tonnes'],
       ['GLO', 'Market for N-fertiliser', 'm24.b', 'M_NFER', 'tonnes'],
       ['GLO', 'Market for Oil seeds', 'm01.e', 'M_OILS', 'tonnes'],
       ['GLO', 'Market for P- and other fertiliser', 'm24.c', 'M_PFER',
        'tonnes'],
       ['GLO', 'Market for Plastics, basic', 'm24.a', 'M_PLAS', 'tonnes'],
       ['GLO', 'Market for Pulp', 'm21.1', 'M_PULP', 'tonnes'],
       ['GLO', 'Market for Wool, silk-worm cocoons', 'm01.o', 'M_WOOL',
        'tonnes']], dtype=object)

In [ ]:
U_without_elec = exio_uagg.copy()
V_without_elec = exio_vagg.copy()
diag_dummy_indices = np.arange(V_without_elec.shape[0])
V_without_elec[diag_dummy_indices, diag_dummy_indices] = 0 #set to zero for principle production so we can sum off diagonal electricity

V_elecmarkets = np.zeros(len(elec_markets)) #These are the totals of the electricity used in a country, as this is the total that a national grid will provide. Format is (N_countries x 1), will be diagonalized in final V' table
U_elecmarkets = np.zeros((exio_uagg.shape[0],len(elec_markets))) #This defines the electricity mix in a national grid. This will be updated with entso data. Format (N_products*N_countries x N_countries)
elec_indices = iot_names['Product code 1'].str.contains('p40.11').values #boolean array with the electricity commodities
elec_martket_product_use = np.zeros((len(elec_markets), exio_uagg.shape[0])) #The input vector for grid electricity for the different activities, as they now draw from the grid instead of directly from producers. Format (N_countries, N_countries*N_products+N_countries)
elec_martket_product_supply = np.zeros((len(elec_markets), exio_uagg.shape[0]))

for i in range(len(elec_markets)): #loop over countries
    U_elecmarkets[elec_indices,i] = exio_uagg[elec_indices,i*164:i*164+164].sum(axis=1)
    V_elecmarkets[i] = U_elecmarkets[elec_indices,i].sum()
    elec_martket_product_use[i,i*164:i*164+164] = exio_uagg[elec_indices,i*164:i*164+164].sum(axis=0)
    elec_martket_product_supply[i,i*164:i*164+164] = V_without_elec[elec_indices,i*164:i*164+164].sum(axis=0)
    V_without_elec[elec_indices,i*164:i*164+164] = 0

V_without_elec[diag_dummy_indices, diag_dummy_indices] = exio_vagg[diag_dummy_indices, diag_dummy_indices] 
U_without_elec[elec_indices,:] = 0 #is the new partial Use table where electricity use from producers has been set to 0

In [270]:
elec_martket_product_supply[1,96+15+28+164]


527.9149686000001

In [271]:
unique_prod_indices = np.unique(prod_names[:,1], return_index=True)[-1]
excl_byproducts = prod_names[unique_prod_indices]
rm_index = []
for i,prod in enumerate(excl_byproducts):
    if 'p40.11.' in prod[2]:
        rm_index.append(i)
excl_byproducts = np.delete(excl_byproducts, np.array(rm_index), axis=0)

In [272]:
print(market_names)
excl_byproducts

[['GLO' 'Market for Aluminium and aluminium products' 'm27.42' 'M_ALUM'
  'tonnes']
 ['GLO' 'Market for Biogas an other gases nec.' 'm40.2.1' 'M_BGAS'
  'tonnes']
 ['GLO' 'Market for N-fertiliser' 'm24.b' 'M_NFER' 'tonnes']
 ['GLO' 'Market for Oil seeds' 'm01.e' 'M_OILS' 'tonnes']
 ['GLO' 'Market for P- and other fertiliser' 'm24.c' 'M_PFER' 'tonnes']
 ['GLO' 'Market for Plastics, basic' 'm24.a' 'M_PLAS' 'tonnes']
 ['GLO' 'Market for Pulp' 'm21.1' 'M_PULP' 'tonnes']
 ['GLO' 'Market for Wool, silk-worm cocoons' 'm01.o' 'M_WOOL' 'tonnes']]


array([['AT', 'Aluminium and aluminium products', 'p27.42', 'C_ALUM',
        'tonnes'],
       ['AT', 'Biogas an other gases nec.', 'p40.2.1', 'C_BGAS',
        'tonnes'],
       ['CY', 'N-fertiliser', 'p24.b', 'C_NFER', 'tonnes'],
       ['GR', 'Oil seeds', 'p01.e', 'C_OILS', 'tonnes'],
       ['CY', 'P- and other fertiliser', 'p24.c', 'C_PFER', 'tonnes'],
       ['MT', 'Plastics, basic', 'p24.a', 'C_PLAS', 'tonnes'],
       ['MT', 'Pulp', 'p21.1', 'C_PULP', 'tonnes'],
       ['AT', 'Wool, silk-worm cocoons', 'p01.o', 'C_WOOL', 'tonnes']],
      dtype=object)

In [273]:
v_market_excl_byproduct = np.zeros(len(excl_byproducts)) #create a vector for the market supply
u_market_excl_byproduct = np.zeros((U_without_elec.shape[0], len(excl_byproducts)))
excl_market_products_use = np.zeros((len(excl_byproducts),U_without_elec.shape[0]))
excl_market_products_supply = np.zeros((len(excl_byproducts),U_without_elec.shape[0]))

U_markets = U_without_elec.copy() #this now will become the final use table (without markets)
V_markets = V_without_elec.copy() #this now will become the final supply table without markets

for i,excl_prod in enumerate(excl_byproducts):
        prod_ind = prod_dic[excl_prod[1]]
        excl_prod_indices = np.where(iot_names.as_matrix()[:,2]==excl_byproducts[i,2])[0]
        u_market_excl_byproduct[excl_prod_indices,i] = exio_vagg[excl_prod_indices,excl_prod_indices]
        v_market_excl_byproduct[i] = u_market_excl_byproduct[:,i].sum()
        
        excl_byprod_countries = prod_names[np.where(prod_names[:,2] == excl_prod[2]),0][0]
        c_index = np.array([country_dic[x] for x in excl_byprod_countries])
        excl_market_products_use[i,:] = exio_uagg[c_index*164+prod_ind,:].sum(axis=0) #every acticity that normally buys the product from a country where this is only a by product now needs to buy this from the market.
        U_markets[c_index*164+prod_ind,:] = 0 #We then set the use from the particular countries to 0 as they only buy from the market.
        
        excl_market_products_supply[i,:] = exio_vagg[c_index*164+prod_ind,:].sum(axis=0) #We now need to move the byproduction in these countries to the market. We can simpy sum over the columns as the production of the other countries in this country will always be 0 and won't affect the sum
        V_markets[c_index*164+prod_ind,:] = 0 #as above for the use but now for the byproduct supply

In [274]:
#here we assemble everything into one Use and one Supply table
exio_dim = exio_uagg.shape[0]
n_elecmarket = len(elec_markets)
n_excl_bp = len(excl_byproducts)
full_dimension =exio_dim + n_elecmarket + n_excl_bp

U_full = np.zeros((full_dimension,full_dimension))
V_full = np.zeros((full_dimension,full_dimension))
#first insert the main use and supply tables
U_full[:exio_dim,:exio_dim] = U_markets
V_full[:exio_dim,:exio_dim] = V_markets

#now insert_electricity markets
U_full[exio_dim:exio_dim+n_elecmarket,:exio_dim] = elec_martket_product_use
U_full[:exio_dim,exio_dim:exio_dim+n_elecmarket] = U_elecmarkets
V_full[exio_dim:exio_dim+n_elecmarket,:exio_dim] = elec_martket_product_supply
V_full[exio_dim:exio_dim+n_elecmarket,exio_dim:exio_dim+n_elecmarket] = np.diag(V_elecmarkets)

#now insert exclusive byproduct markets
U_full[exio_dim+n_elecmarket:,:exio_dim] = excl_market_products_use
U_full[:exio_dim,exio_dim+n_elecmarket:] = u_market_excl_byproduct

V_full[exio_dim+n_elecmarket:,:exio_dim] = excl_market_products_supply
V_full[exio_dim+n_elecmarket:,exio_dim+n_elecmarket:] = np.diag(v_market_excl_byproduct)




In [275]:
print(elec_markets)
print(market_names)
#create market electricity product names

[['AU' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['AT' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['BE' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['BR' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['BG' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['CA' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['CN' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['CY' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['CZ' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['DK' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['EE' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['FI' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['FR' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['DE' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['GR' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['HU' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['HR' 'Market for Electricity' 'm40.11' 'M_ELEC' 'TJ']
 ['IN' 'Market for Electricity' 'm40.11' 'M_ELEC

$Z = U = $

In [276]:
def make_IOT(U_full, V_full):
    Z_full = U_full - V_full + np.diag(np.diag(V_full))
    x_dummy = np.diag(V_full).copy()
    x_dummy[x_dummy == 0] = 1
    A_full = Z_full/x_dummy
    return Z_full, A_full
Z_full, A_full = make_IOT(U_full, V_full)


* check for every non zero entry in entsoe if it has principle production exiobase
* update information

In [277]:
#Z_e_update, A_e_update = update_electricity_mix()

def update_electricity_mix(entso_u, entso_v, entso_act, entso_prod, U,V):
    
    country_ind = country_dic[country_code]
    return U_update, V_update




In [278]:
bentso_prods = json.load(open('/home/jakobs/Documents/IndEcol/BONSAI/bentso-data/entsoe-beebee-2016/entsoe-products.json'))
bentso_acts = json.load(open('/home/jakobs/Documents/IndEcol/BONSAI/bentso-data/entsoe-beebee-2016/entsoe-activities.json'))

In [279]:
bentso_use = np.load('/home/jakobs/Documents/IndEcol/BONSAI/bentso-data/entsoe-beebee-2016/entsoe-use.npy')
bentso_supply = np.load('/home/jakobs/Documents/IndEcol/BONSAI/bentso-data/entsoe-beebee-2016/entsoe-supply.npy')

In [280]:
bentso_acts

[['AT', 'Biomass'],
 ['AT', 'Fossil Brown coal/Lignite'],
 ['AT', 'Fossil Coal-derived gas'],
 ['AT', 'Fossil Gas'],
 ['AT', 'Fossil Hard coal'],
 ['AT', 'Fossil Oil'],
 ['AT', 'Fossil Oil shale'],
 ['AT', 'Fossil Peat'],
 ['AT', 'Geothermal'],
 ['AT', 'Grid'],
 ['AT', 'Hydro Pumped Storage'],
 ['AT', 'Hydro Run-of-river and poundage'],
 ['AT', 'Hydro Water Reservoir'],
 ['AT', 'Marine'],
 ['AT', 'Nuclear'],
 ['AT', 'Other'],
 ['AT', 'Other renewable'],
 ['AT', 'Solar'],
 ['AT', 'Waste'],
 ['AT', 'Wind Offshore'],
 ['AT', 'Wind Onshore'],
 ['BA', 'Biomass'],
 ['BA', 'Fossil Brown coal/Lignite'],
 ['BA', 'Fossil Coal-derived gas'],
 ['BA', 'Fossil Gas'],
 ['BA', 'Fossil Hard coal'],
 ['BA', 'Fossil Oil'],
 ['BA', 'Fossil Oil shale'],
 ['BA', 'Fossil Peat'],
 ['BA', 'Geothermal'],
 ['BA', 'Grid'],
 ['BA', 'Hydro Pumped Storage'],
 ['BA', 'Hydro Run-of-river and poundage'],
 ['BA', 'Hydro Water Reservoir'],
 ['BA', 'Marine'],
 ['BA', 'Nuclear'],
 ['BA', 'Other'],
 ['BA', 'Other renewable'

In [281]:
print(32*12)
print(672/32.)

384
21.0


In [282]:
pd.DataFrame(columns=np.array(bentso_acts).T.tolist(), index=np.array(bentso_prods).T.tolist(), data=bentso_use)

AT                            \
                                         Biomass Fossil Brown coal/Lignite   
AT Biomass                          3.651084e+10                       0.0   
   Fossil Brown coal/Lignite        0.000000e+00                       0.0   
   Fossil Coal-derived gas          0.000000e+00                       0.0   
   Fossil Gas                       0.000000e+00                       0.0   
   Fossil Hard coal                 0.000000e+00                       0.0   
   Fossil Oil                       0.000000e+00                       0.0   
   Fossil Oil shale                 0.000000e+00                       0.0   
   Fossil Peat                      0.000000e+00                       0.0   
   Geothermal                       0.000000e+00                       0.0   
   Grid                             0.000000e+00                       0.0   
   Hydro Pumped Storage             0.000000e+00                       0.0   
   Hydro Run-of-river and poundage  0.000000e+00                       0.0   
   Hydro Water Reservoir            0.000000e+00                       0.0   
   Marine                           0.000000e+00                       0.0   
   Nuclear                          0.000000e+00                       0.0   
   Other                            0.000000e+00                       0.0   
   Other renewable                  0.000000e+00                       0.0   
   Solar                            0.000000e+00                       0.0   
   Waste                            0.000000e+00                       0.0   
   Wind Offshore                    0.000000e+00                       0.0   
   Wind Onshore                     0.000000e+00                       0.0   
BA Biomass                          0.000000e+00                       0.0   
   Fossil Brown coal/Lignite        0.000000e+00                       0.0   
   Fossil Coal-derived gas          0.000000e+00                       0.0   
   Fossil Gas                       0.000000e+00                       0.0   
   Fossil Hard coal                 0.000000e+00                       0.0   
   Fossil Oil                       0.000000e+00                       0.0   
   Fossil Oil shale                 0.000000e+00                       0.0   
   Fossil Peat                      0.000000e+00                       0.0   
   Geothermal                       0.000000e+00                       0.0   
...                                          ...                       ...   
SI Hydro Water Reservoir            0.000000e+00                       0.0   
   Marine                           0.000000e+00                       0.0   
   Nuclear                          0.000000e+00                       0.0   
   Other                            0.000000e+00                       0.0   
   Other renewable                  0.000000e+00                       0.0   
   Solar                            0.000000e+00                       0.0   
   Waste                            0.000000e+00                       0.0   
   Wind Offshore                    0.000000e+00                       0.0   
   Wind Onshore                     0.000000e+00                       0.0   
SK Biomass                          0.000000e+00                       0.0   
   Fossil Brown coal/Lignite        0.000000e+00                       0.0   
   Fossil Coal-derived gas          0.000000e+00                       0.0   
   Fossil Gas                       0.000000e+00                       0.0   
   Fossil Hard coal                 0.000000e+00                       0.0   
   Fossil Oil                       0.000000e+00                       0.0   
   Fossil Oil shale                 0.000000e+00                       0.0   
   Fossil Peat                      0.000000e+00                       0.0   
   Geothermal                       0.000000e+00                       0.0   
   Grid                             0.000000e+00                    

In [283]:
set(country_concord['bentso_Country_code'].unique()).difference(set(europe_exio))

{'TR', nan}

In [284]:
country_concord

,exiobase_country_code,bentso_Country_code,Geonames_code
0,AU,NaN,2077456
1,AT,AT,2782113
2,BE,BE,2802361
3,BR,NaN,3469034
4,BG,BG,732800
5,CA,NaN,6251999
6,CN,NaN,1814991
7,CY,NaN,146669
8,CZ,CZ,3077311
9,DK,DK,2623032


In [285]:
prod_cor = pd.read_csv('/home/jakobs/Documents/IndEcol/BONSAI/Correspondence-tables/final_tables/tables/exiobase_to_bentso_activities.csv')

In [286]:
len(prod_cor['Bentso technology'].dropna())

18

In [287]:
ACTIVITY_MAPPING = {
    'Fossil Hard coal': 'Production of electricity by coal',
    'Fossil Brown coal/Lignite': 'Production of electricity by coal',
    'Fossil Gas': 'Production of electricity by gas',
    'Fossil Coal-derived gas': 'Production of electricity by gas',
    'Nuclear': 'Production of electricity by nuclear',
    'Hydro Pumped Storage': 'Production of electricity by hydro',
    'Hydro Run-of-river and poundage': 'Production of electricity by hydro',
    'Hydro Water Reservoir': 'Production of electricity by hydro',
    'Wind Offshore': 'Production of electricity by wind',
    'Wind Onshore': 'Production of electricity by wind',
    'Fossil Oil': 'Production of electricity by petroleum and other oil derivatives',
    'Fossil Oil shale': 'Production of electricity by petroleum and other oil derivatives',
    'Biomass': 'Production of electricity by biomass and waste',
    'Fossil Peat': 'Production of electricity by coal',
    'Waste': 'Production of electricity by biomass and waste',
    'Solar': 'Production of electricity by solar thermal',
    'Other renewable': 'Production of electricity by tide, wave, ocean',
    'Geothermal': 'Production of electricity by Geothermal',
    'Other': 'Production of electricity nec',
    'Marine': 'Production of electricity nec',
    'Grid': "Market for Electricity",
}
FLOW_MAPPING = {
    'Fossil Hard coal': 'Electricity by coal',
    'Fossil Brown coal/Lignite': 'Electricity by coal',
    'Fossil Gas': 'Electricity by gas',
    'Fossil Coal-derived gas': 'Electricity by gas',
    'Nuclear': 'Electricity by nuclear',
    'Hydro Pumped Storage': 'Electricity by hydro',
    'Hydro Run-of-river and poundage': 'Electricity by hydro',
    'Hydro Water Reservoir': 'Electricity by hydro',
    'Wind Offshore': 'Electricity by wind',
    'Wind Onshore': 'Electricity by wind',
    'Fossil Oil': 'Electricity by petroleum and other oil derivatives',
    'Fossil Oil shale': 'Electricity by petroleum and other oil derivatives',
    'Biomass': 'Electricity by biomass and waste',
    'Fossil Peat': 'Electricity by coal',
    'Waste': 'Electricity by biomass and waste',
    'Solar': 'Electricity by solar thermal',
    'Other renewable': 'Electricity by tide; wave; ocean',
    'Geothermal': 'Electricity by Geothermal',
    'Other': 'Electricity nec',
    'Marine': 'Electricity nec',
    'Grid': "Electricity",
}



In [288]:
np.unique([ACTIVITY_MAPPING[x] for x in ACTIVITY_MAPPING.keys()])

array(['Market for Electricity',
       'Production of electricity by Geothermal',
       'Production of electricity by biomass and waste',
       'Production of electricity by coal',
       'Production of electricity by gas',
       'Production of electricity by hydro',
       'Production of electricity by nuclear',
       'Production of electricity by petroleum and other oil derivatives',
       'Production of electricity by solar thermal',
       'Production of electricity by tide, wave, ocean',
       'Production of electricity by wind',
       'Production of electricity nec'], dtype='<U64')

In [289]:
np.unique([x for x in ACTIVITY_MAPPING.keys()])

array(['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas',
       'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Oil shale',
       'Fossil Peat', 'Geothermal', 'Grid', 'Hydro Pumped Storage',
       'Hydro Run-of-river and poundage', 'Hydro Water Reservoir',
       'Marine', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste',
       'Wind Offshore', 'Wind Onshore'], dtype='<U31')

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00798212, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

array([[1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 4]])

In [393]:
def my_function(x,y):
    x[5] = 100
    y[0:2] = 1000
    return x,y

In [396]:
a = np.arange(10)
b = a.copy()

c,d = my_function(a,b)

print(a)
print(b)
print(c)
print(d)
print(a==c)
print(b==d)

b[5] = 0
print(b)
print(a)



[  0   1   2   3   4 100   6   7   8   9]
[1000 1000    2    3    4    5    6    7    8    9]
[  0   1   2   3   4 100   6   7   8   9]
[1000 1000    2    3    4    5    6    7    8    9]
[ True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True]
[1000 1000    2    3    4    0    6    7    8    9]
[  0   1   2   3   4 100   6   7   8   9]


In [419]:
exio_u.shape

(9600, 7872)

'my_function'

In [422]:
1*3e5/60/60/24.

3.472222222222222